# Các thư viện cần thiết

In [1]:
%pip install unidecode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import time
from unidecode import unidecode

In [3]:
# Đường dẫn đến ChromeDriver
DRIVER_PATH = 'C:\\chromedriver\\chromedriver.exe'

# Tạo đối tượng Service
service = Service(DRIVER_PATH)

# Khởi tạo ChromeDriver với service
driver =None



In [4]:
def scroll_and_load_more(driver, max_scrolls=3):
    scroll_count = 0
    while scroll_count < max_scrolls:
        # Cuộn xuống cuối trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Chờ trang tải đầy đủ
        
        # Tìm nút "Tải thêm" và nhấp vào nếu có
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-primary"))
            )
            actions = ActionChains(driver)
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            time.sleep(0.5)  # Chờ để cuộn đến nút "Tải thêm"

            # Cuộn trang thêm một đoạn để đảm bảo nút không bị che khuất
            driver.execute_script("window.scrollBy(0, -150);")
            time.sleep(0.5)  # Chờ để cuộn

            actions.move_to_element(load_more_button).click().perform()
            time.sleep(3)  # Chờ để tải thêm dữ liệu
        except Exception as e:
            print(f"Exception encountered: {e}")
            break
        
        scroll_count += 1


# Cào link

In [5]:
##danh sách để lưu trữ các link
driver = webdriver.Chrome(service=service)
driver.get("https://guland.vn/mua-ban-bat-dong-san")
scroll_and_load_more(driver, max_scrolls=20)

# Danh sách để lưu trữ các link
arr = []
linkSelector = ".sdb-image-wrap "  # CSS selector cho các thẻ <a> trong sdb-image-wrap

# Lấy các liên kết
links = driver.find_elements(By.CSS_SELECTOR, linkSelector)
for link in links:
    arr.append(link.get_attribute('href'))

# Tạo DataFrame từ danh sách các link
df = pd.DataFrame(arr, columns=['links'])
print("Hoàn thành lấy dữ liệu!")
print(df)

# Đóng trình duyệt
driver.quit()


Hoàn thành lấy dữ liệu!
                                                 links
0    https://guland.vn/post/ban-dat-eatu-sau-truong...
1    https://guland.vn/post/ban-lo-lo-dat-520-tc-10...
2    https://guland.vn/post/ban-dat-eatu-ful-520-fu...
3    https://guland.vn/post/nguyen-chi-thanh-ban-da...
4    https://guland.vn/post/ban-dat-1596m2-19-ty-ta...
..                                                 ...
940  https://guland.vn/post/ban-ban-dat-tho-cu-874m...
941  https://guland.vn/post/ban-ban-nha-rieng-109m2...
942  https://guland.vn/post/ban-ban-dat-tho-cu-767m...
943  https://guland.vn/post/ban-ban-dat-tho-cu-1092...
944  https://guland.vn/post/ban-ban-dat-tho-cu-768m...

[945 rows x 1 columns]


# Xuất file link

In [6]:
df.to_csv('data\\guland_links.csv', index=False)

# Check xem dữ liệu có bị trùng lặp không

In [7]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
940    False
941    False
942    False
943    False
944    False
Length: 945, dtype: bool

# Test thử xem có lấy dữ liệu không

In [8]:
frame = pd.read_csv('data\\guland_links.csv')
frame.shape
frame.head(20)

,links
0,https://guland.vn/post/ban-dat-eatu-sau-truong...
1,https://guland.vn/post/ban-lo-lo-dat-520-tc-10...
2,https://guland.vn/post/ban-dat-eatu-ful-520-fu...
3,https://guland.vn/post/nguyen-chi-thanh-ban-da...
4,https://guland.vn/post/ban-dat-1596m2-19-ty-ta...
5,https://guland.vn/post/qua-hoi-voi-198tr-so-hu...
6,https://guland.vn/post/ban-dat-200m2-335-trieu...
7,https://guland.vn/post/ban-lo-2-mat-tien-ama-p...
8,https://guland.vn/post/ban-dat-tang-nha-mt-vo-...
9,https://guland.vn/post/ban-nha-299m2-125-ty-ta...


# Cào dữ liệu

In [29]:
data_dict = {
    'dia_chi': [],
    'ma_tin': [],
    'gia': [],
    'dien_tich': [],
    'gia_tren_metvuong': [],
    'ten_du_an': [],
    'loai_bds': [],
    'chieu_ngang': [],
    'chieudai': [],
    'phong_ngu': [],
    'phong_tam': [],
    'so_tang': [],
    'vi_tri': [],
    'huong_cua': [],
    'duong_vao_hem': [],
    'loai_duong': []
}
items =['Tên dự án:','Loại BĐS:','Chiều ngang:','Chiều dài:','Số phòng ngủ:','Số phòng tắm:',
        'Số tầng','Vị trí:',"Hướng cửa chính",'Đường/hẻm vào rộng','Loại đường'
        ]

In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from requests_html import HTML
def getdata(link) ->dict:
    # Cấu hình Selenium WebDriver
    # Tạo instance của trình điều khiển Chrome
    driver = webdriver.Chrome(service=service)
        # Mở trang web        
    driver.get(link)
        # Lấy nội dung HTML của trang
    html_text = driver.page_source
    driver.quit()  # Đóng trình duyệt sau khi lấy dữ liệu
        # Phân tích cú pháp HTML để trích xuất dữ liệu
    r_html = HTML(html=html_text)
    if len(r_html.find(".notfound-message")) == 0:
        arr = []
        r_table = r_html.find(".sdb-picker-site")
        if len(r_table) > 0:
            # test là toàn bộ mã html của trang đang truy xuất
            test = r_table[0]
            main_info=test.find('.dtl-prc__row')
            tmp=test.find('.dtl-stl__row')
            dia_chi=tmp[0]
            ma_tin_tmp = test.find('.dtl-stl__row')[1]
            ma_tin_text = ma_tin_tmp.text
            
            # Tìm vị trí của chuỗi "Mã tin:" để lấy mã tin
            start_index = ma_tin_text.find(":")
            end_index= ma_tin_text.find("Cập nhật")
            ma_tin = ma_tin_text[start_index+2 :end_index].strip()
            tmp1=test.find('.dtl-inf__row')
            inf = tmp1[0].text
            arr.append(main_info[0].text)  # các thông tin chính các căn nhà, gồm có tiền, diện tích
            arr.append(dia_chi.text)# địa chỉ của căn nhà
            arr.append(ma_tin)# mã tin
            arr.append(inf)# các thông tin phụ của căn nhà
        else:
            return []
        str1 = arr
        str1 = [arr]
        temp = arr[0].split('\n')#tách dữ liệu chính
        tmp=arr[3].split('\n')# tách các dữ liệu phụ
        idx = []
        idx.append(arr[1])
        idx.append(arr[2])
        idx.append(temp[0])#gia
        idx.append(temp[1])#dien tich
        idx.append(temp[2])#gia tren m^2
        for i in range(0, len(tmp), 2):
            if i + 1 < len(tmp):
                idx.append([tmp[i]+" "+ tmp[i + 1]])
            else:
                idx.append([tmp[i]])
        return idx
    else:
        return []




In [41]:

with open('data\\guland_data.json', 'w', encoding='utf-8') as file:
    file.write('[')  # Bắt đầu một json array
    first = True
    
    for link in frame['links']:
        try:
            print(f"Processing link: {link}")
            data = getdata(link)
            dem = 0 
            if len(data) > 0:
                data_dict['dia_chi'].append(data[0])
                data_dict['ma_tin'].append(data[1])
                data_dict['gia'].append(data[2])
                data_dict['dien_tich'].append(data[3])
                data_dict['gia_tren_metvuong'].append(data[4])
                for item in items:
                    mang = []
                    flag = 0
                    for i in data:
                        if len(i) > 0 and item in i[0]:
                            mang.append(i[0].split(':')[1])
                            flag = 1
                            break
                    dem += 1
                    key = None
                    if dem == 1: key = 'ten_du_an'
                    elif dem == 2: key = 'loai_bds'
                    elif dem == 3: key = 'chieu_ngang'
                    elif dem == 4: key = 'chieudai'
                    elif dem == 5: key = 'phong_ngu'
                    elif dem == 6: key = 'phong_tam'
                    elif dem == 7: key = 'so_tang'
                    elif dem == 8: key = 'vi_tri'
                    elif dem == 9: key = 'huong_cua'
                    elif dem == 10: key = 'duong_vao_hem'
                    elif dem == 11: key = 'loai_duong'
                    
                    if key:
                        if flag == 1:
                            data_dict[key] += mang
                        else:
                            data_dict[key].append(None)
        except Exception as e:
            print(f"Error processing link: {link}")
            print(f"Exception: {e}")
    
    # Write to JSON file
        if data is not None:
            if not first:
                file.write(',\n')
            json.dump(data_dict, file, ensure_ascii=False, indent=4)
            first = False
            data_dict = {key: [] for key in data_dict.keys()}  # Reset data_dict

    file.write(']')

Processing link: https://guland.vn/post/ban-dat-eatu-sau-truong-ngo-may-5520-gia-chot-725-504827
Processing link: https://guland.vn/post/ban-lo-lo-dat-520-tc-100-eatu-gia-ban-800-chot-cung-khong-thuong-luong-507289
Processing link: https://guland.vn/post/ban-dat-eatu-ful-520-full-tc-gia-ban-790-xa-lo-90-trieu-so-voi-gia-mua-vao-492268
Processing link: https://guland.vn/post/nguyen-chi-thanh-ban-dat-2-lo-lien-ke-1230-tho-cu-300-gia-ban-14-tlcc-821496
Processing link: https://guland.vn/post/ban-dat-1596m2-19-ty-tai-phuong-ea-tam-thanh-pho-buon-ma-thuot-897859


# Cào và lưu json

# Đọc file json

In [42]:
with open('data\\guland_data.json', 'r',encoding='utf-8') as file:
    json_data = json.load(file)

df_items = pd.json_normalize(json_data)
df_items

,dia_chi,ma_tin,gia,dien_tich,gia_tren_metvuong,ten_du_an,loai_bds,chieu_ngang,chieudai,phong_ngu,phong_tam,so_tang,vi_tri,huong_cua,duong_vao_hem,loai_duong
0,"[Xã Ea Tu, Thành phố Buôn Ma Thuột, Đắk Lắk, X...",[504827],[725 triệu],[110m²],[6.59 triệu /m²],[None],"[ Đất ở, đất thổ cư]",[ 5.5m],[ 20m],[None],[None],[None],[ Trong hẻm],[ Tây],[ 5m],[ Đường bê tông]
1,"[Xã Ea Tu, Thành phố Buôn Ma Thuột, Đắk Lắk]",[507289],[800 triệu],[100m²],[8 triệu /m²],[None],"[ Đất ở, đất thổ cư]",[ 5m],[ 20m],[None],[None],[None],[ Trong hẻm],[ Đông],[ 4m],[ Đường đất]
2,"[Xã Ea Tu, Thành phố Buôn Ma Thuột, Đắk Lắk]",[492268],[790 triệu],[100m²],[7.9 triệu /m²],[None],"[ Đất ở, đất thổ cư]",[ 5m],[ 20m],[None],[None],[None],[ Trong hẻm],[ Đông],[ 4m],[ Đường bê tông]
3,"[Phường Tân An, Thành phố Buôn Ma Thuột, Đắk Lắk]",[821496],[14 tỷ],[300m²],[46.67 triệu /m²],[None],"[ Đất ở, đất thổ cư]",[ 12m],[ 30m],[None],[None],[None],[ Đường chính],[ Tây Bắc],[ 30m],[ Đường nhựa]
